In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import dash_table
import plotly.express as px
import dash_leaflet as dl
import pandas as pd
import base64
from pymongo import MongoClient
from pymongo.errors import PyMongoError

# CRUD class definition
class CRUD:
    def __init__(self, db_name, collection_name, mongo_uri):
        self.db_name = db_name
        self.collection_name = collection_name
        self.mongo_uri = mongo_uri
        try:
            self.client = MongoClient(mongo_uri)
            self.db = self.client[db_name]
            self.collection = self.db[collection_name]
            print("MongoDB connection successful.")
        except PyMongoError as e:
            print(f"Error connecting to MongoDB: {e}")

    def create(self, document):
        try:
            result = self.collection.insert_one(document)
            return True
        except PyMongoError as e:
            print(f"Error inserting document: {e}")
            return False

    def read(self, query, limit=None):
        try:
            documents = self.collection.find(query)
            if limit is not None:
                documents = documents.limit(limit)
            return list(documents)
        except PyMongoError as e:
            print(f"Error reading documents: {e}")
            return []

    def update(self, query, update_data):
        try:
            result = self.collection.update_many(query, {"$set": update_data})
            return result.modified_count
        except PyMongoError as e:
            print(f"Error updating documents: {e}")
            return 0

    def delete(self, query):
        try:
            result = self.collection.delete_many(query)
            return result.deleted_count
        except PyMongoError as e:
            print(f"Error deleting documents: {e}")
            return 0

# MongoDB connection credentials
username = "aacuser"
password = "Harley"
host = "nv-desktop-services.apporto.com"
port = 30466
database = "AAC"
collection = "animals"


# Create the connection string for MongoDB
mongo_uri = f"mongodb://{username}:{password}@{host}:{port}/{database}?authSource=admin"
        
# Connect to MongoDB
client = MongoClient(mongo_uri)
db = client[database]
col = db[collection]

# Instantiate CRUD class
shelter = CRUD(db_name=database, collection_name=collection, mongo_uri=mongo_uri)

# Fetch sample data
query = {
    'breed': {'$in': ['Siberian Husky', 'Alaskan Malamute', 'Rottweiler', 'German Shepherd']},
    'sex_upon_outcome': 'Intact Male',
    'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
data = shelter.read({}, limit=100)  # Retrieve 100 documents
print("Data retrieved:", data)  # Display the retrieved data

# Load data into a DataFrame
df = pd.DataFrame.from_records(data)

# Remove the '_id' column if necessary
df.drop(columns=['_id'], inplace=True)

# Load and encode the Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'
try:
    with open(image_filename, 'rb') as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
except FileNotFoundError:
    encoded_image = ''
    print(f"Image file {image_filename} not found.")
    
data = list(col.find(query))
df = pd.DataFrame.from_records(data)
    
# Debug: Print the fetched data and column names
print("Fetched Data:", data)
print("DataFrame Columns:", df.columns)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the Dash app
app.layout = html.Div([
    html.Div([
        # Grazioso Salvare's logo
        html.Img(
            src='data:image/png;base64,{}'.format(encoded_image) if encoded_image else '',
            style={'width': '200px', 'display': 'block', 'margin': 'auto'}
        ),
        html.H1("SNHU CS-340 Vincent Yanez Dashboard 2024", style={'textAlign': 'center', 'color': 'green'}),
        
        # Dropdown for rescue type
        dcc.RadioItems(
            id='rescue-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset'
        )
    ], style={'padding': '20px'}),

    html.Div([
        # Data table
        dash_table.DataTable(
            id='data-table',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            row_selectable='single',
            selected_rows=[0],  # Default to selecting the first row
            page_size=10,  # Limit the number of rows displayed per page
            sort_action='native',  # Enable sorting
            filter_action='native',  # Enable filtering
            style_table={'overflowX': 'auto'},
            style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'},
            style_data={'color': 'black'}
        ),
        
        html.Br(),
        html.Hr(),
        
        #Graphs and map
        html.Div(className='row', style={'display': 'flex'}, children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6')
        ])
    ], style={'backgroundColor': 'white'})
])

# Define the callback function
@app.callback(
    [Output('data-table', 'data'),
     Output('graph-id', 'children'),
     Output('map-id', 'children')],
    [Input('rescue-type', 'value')]
)
def update_dashboard(rescue_type):
    # Define the rescue type filters
    rescue_type_map = {
        'Water Rescue': {
            'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
            'sex': 'Intact Female',
            'age_range': (26, 156)
        },
        'Mountain or Wilderness Rescue': {
            'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
            'sex': 'Intact Male',
            'age_range': (26, 156)
        },
        'Disaster or Individual Tracking': {
            'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
            'sex': 'Intact Male',
            'age_range': (20, 300)
        }
    }

    if rescue_type == 'Reset':
        # Show all data if 'Reset' is selected
        filtered_df = df
    else:
        # Apply the filter based on the selected rescue type
        filters = rescue_type_map.get(rescue_type, {})
        breed_filter = filters.get('breeds', [])
        sex_filter = filters.get('sex', '')
        age_min, age_max = filters.get('age_range', (0, float('inf')))

        filtered_df = df[
            (df['breed'].isin(breed_filter)) &
            (df['sex_upon_outcome'] == sex_filter) &
            (df['age_upon_outcome_in_weeks'] >= age_min) &
            (df['age_upon_outcome_in_weeks'] <= age_max)
        ]

    # Update the data table
    table_data = filtered_df.to_dict('records')
    
    # Generate the graph
    fig = px.pie(filtered_df, names='breed', title='Preferred Animals')
    graph_children = dcc.Graph(figure=fig)
    
    # Generate the map
    if len(filtered_df) == 0:
        map_children = html.P("No data available for selected filter.")
    else:
        row = filtered_df.iloc[0]  # Use the first row as an example
        map_children = dl.Map(style={'width': '1000px', 'height': '500px'},
                              center=[30.75, -97.48], zoom=10, children=[
                                  dl.TileLayer(id="base-layer-id"),
                                  dl.Marker(
                                      position=[row['location_lat'], row['location_long']],  # Adjust column names
                                      children=[
                                          dl.Tooltip(f"Name: {row['name']}"),  # Show animal's name in tooltip
                                          dl.Popup([
                                              html.H1(f"Name: {row['name']}"),  # Show animal's name in popup
                                              html.P(f"ID: {row['animal_id']}"),  # Show animal ID in popup
                                              html.P(f"Outcome: {row['outcome_type']}")  # Show outcome type in popup
                                          ])
                                      ]
                                  )
                              ])
    
    return table_data, graph_children, map_children


# Run the server
if __name__ == '__main__':
  #app.run_server(debug=True)
    app.run_server(port=8990)





SyntaxError: invalid syntax (1565569052.py, line 85)